# 功能丰富的推荐系统

交互数据是用户偏好和兴趣的最基本指示。它在以前引入的模型中起着关键作用。然而，交互数据通常非常稀疏，有时会有噪声。为了解决这个问题，我们可以将诸如项目特征、用户配置文件，甚至是交互发生在哪个上下文中的辅助信息集成到推荐模型中。利用这些特性有助于提出建议，因为这些特性可以有效预测用户的兴趣，尤其是在缺少交互数据时。因此，推荐模型还必须能够处理这些特性，并为模型提供一些内容/上下文感知。为了演示这种类型的推荐模型，我们引入了另一项在线广告推荐点击率（CTR）任务：:cite:`McMahan.Holt.Sculley.ea.2013`，并提供了匿名广告数据。有针对性的广告服务已经引起了广泛的关注，通常被设计成推荐引擎。推荐符合用户个人品味和兴趣的广告对于提高点击率非常重要。


数字营销人员使用在线广告向客户展示广告。点击率是一个衡量广告主在每一次印象中收到的广告点击次数的指标，它用公式计算的百分比表示：

$$ \text{CTR} = \frac{\#\text{Clicks}} {\#\text{Impressions}} \times 100 \% .$$

点击率是指示预测算法有效性的重要信号。点击率预测是一项预测网站被点击的可能性的任务。CTR预测模型不仅可用于目标广告系统，也可用于一般项目（如电影、新闻、产品）推荐系统、电子邮件活动，甚至搜索引擎。它还与用户满意度、转化率密切相关，有助于设定活动目标，因为它可以帮助广告商设定现实的期望。


In [ ]:
%load ../utils/djl-imports

import ai.djl.training.dataset.Record;
import com.google.gson.Gson;

## 在线广告数据集

随着互联网和移动技术的长足进步，在线广告已成为互联网行业的一项重要收入来源，并产生了绝大部分收入。展示相关广告或激起用户兴趣的广告非常重要，这样，临时访客就可以转化为付费客户。我们介绍的数据集是一个在线广告数据集。它由34个字段组成，第一列表示目标变量，指示是否单击了广告（1）或未单击广告（0）。所有其他列都是分类特征。这些列可能表示广告id、站点或应用程序id、设备id、时间、用户配置文件等。由于匿名化和隐私问题，这些特征的真正语义尚未公开。

以下代码从服务器下载数据集并将其保存到本地数据文件夹中。


In [ ]:
var url = "http://d2l-data.s3-accelerate.amazonaws.com/ctr.zip";
ZipUtils.unzip(new URL(url).openStream(), Paths.get("./"));

有一个训练集和一个测试集，分别由15000和3000个样本/行组成。

## 数据集封装

为了方便数据加载，我们实现了一个`CTRDataset`，它从CSV文件加载广告数据集，并且可以由`DataLoader`使用。


In [ ]:
public class CTRDataset extends ArrayDataset {

    private boolean prepared;
    private NDManager manager = Engine.getInstance().newBaseManager();
    private List<Long[]> oneHotFeatures;
    private List<Float> labelList;

    private CTRDataset(Builder builder) {
        super(builder);
        this.oneHotFeatures = builder.oneHotFeatures;
        this.labelList = builder.label;
    }

    @Override
    public void prepare(Progress progress) throws IOException {
        if (prepared) {
            return;
        }
        data = new NDArray[oneHotFeatures.size()];
        labels = new NDArray[labelList.size()];
        for (int i = 0; i < data.length; i++) {
            data[i] = manager.create(Arrays.stream(oneHotFeatures.get(i)).mapToLong(Long::longValue).toArray());
            labels[i] = manager.create(labelList.get(i));
        }
        prepared = true;
    }

    /**
     * {@inheritDoc}
     */
    @Override
    public Record get(NDManager manager, long index) {
        NDList datum = new NDList();
        NDList label = new NDList();

        datum.add(data[(int) index]);
        if (labels != null) {
            label.add(labels[(int) index]);
        }
        datum.attach(manager);
        label.attach(manager);
        return new Record(datum, label);
    }


    public static Builder builder() {
        return new Builder();
    }

    public static final class Builder extends BaseBuilder<Builder> {

        private long numFeatures;
        private long featureThreshold;
        private String fileName;
        // feature id, category String, category code
        private Map<Long, Map<String, Long>> featureMap = new ConcurrentHashMap<>();
        // feature id, category String, category count
        private Map<Long, Map<String, Long>> featureCount = new ConcurrentHashMap<>();
        private Map<Long, Long> defaultValues = new ConcurrentHashMap<>();
        private List<String[]> features = new ArrayList<>();
        private List<Float> label = new ArrayList<>();
        private Long[] fieldDim;
        private Long[] offset;
        private List<Long[]> oneHotFeatures = new ArrayList<>();
        private String outputDir;

        Builder() {
        }

        @Override
        protected Builder self() {
            return this;
        }

        public Builder setFileName(String fileName) {
            this.fileName = fileName;
            return this;
        }

        public Builder optNumFeatures(long numFeatures) {
            this.numFeatures = numFeatures;
            return this;
        }

        public Builder optFeatureThreshold(long featureThreshold) {
            this.featureThreshold = featureThreshold;
            return this;
        }

        public Builder optMapOutputDir(String outputDir) {
            this.outputDir = outputDir;
            return this;
        }

        CTRDataset build() throws IOException {

            try (BufferedReader reader = Files.newBufferedReader(Paths.get(this.fileName))) {
                String line;
                while ((line = reader.readLine()) != null) {
                    String[] record = line.trim().split("\t");
                    if (record.length != this.numFeatures + 1) {
                        continue;
                    }
                    label.add(Float.parseFloat(record[0]));
                    for (int i = 1; i < numFeatures + 1; i++) {
                        Map<String, Long> count = featureCount.computeIfAbsent((long) i, k -> new ConcurrentHashMap<>());
                        // 字符串的增量计数
                        count.merge(record[i], 1L, Long::sum);
                    }
                    features.add(Arrays.copyOfRange(record, 1, record.length));
                }
            }
            fieldDim = new Long[(int) numFeatures];
            offset = new Long[(int) numFeatures];
            // 减少 class 频率
            for (long i = 1L; i < numFeatures + 1; i++) {
                featureCount.get(i).values().removeIf(value -> value < featureThreshold);
                Map<String, Long> reducedFeatures = featureCount.get(i);
                Map<String, Long> featureIndex = new ConcurrentHashMap<>();
                long index = 0;
                for (String feature : reducedFeatures.keySet()) {
                    featureIndex.put(feature, index);
                    index++;
                }
                featureMap.put(i, featureIndex);
                defaultValues.put(i, (long) featureIndex.size());
                fieldDim[(int) i - 1] = (long) featureIndex.size() - 1;
            }
            long sum = 0;
            for (int i = 0; i < fieldDim.length; i++) {
                offset[i] = sum;
                sum += fieldDim[i];
            }

            for (String[] feature : features) {
                Long[] oneHot = new Long[feature.length];
                for (int i = 0; i < oneHot.length; i++) {
                    oneHot[i] = featureMap.get((long) i + 1).getOrDefault(feature[i], defaultValues.get((long) i + 1)) + offset[i];
                }
                oneHotFeatures.add(oneHot);
            }
            // 保存特征映射和默认值以进行推断
            if (outputDir != null) {
                saveMap(featureMap, outputDir, "feature_map.json");
                saveMap(defaultValues, outputDir, "defaults.json");
            }

            return new CTRDataset(this);
        }

        private void saveMap(Map map, String outputDir, String fileName) throws IOException {
            Gson gson = new Gson();
            FileWriter writer = new FileWriter(outputDir + "/" + fileName);
            gson.toJson(map, writer);
            writer.flush();
            writer.close();
        }

    }
}

下面的示例加载训练数据并打印出第一条记录。我们还需要保存特征映射和默认值以进行推断。


In [ ]:
CTRDataset data = CTRDataset.builder()
                .optFeatureThreshold(4)
                .optNumFeatures(34)
                .setFileName("./ctr/train.csv")
                .optMapOutputDir("./")
                .setSampling(1, true)
                .build();
data.prepare();
NDManager manager = NDManager.newBaseManager();
Record record = data.get(manager, 0);
System.out.println(record.getData().singletonOrThrow());
System.out.println(record.getLabels().singletonOrThrow());

可以看出，所有34个字段都是分类特征。每个值表示对应条目的一个热索引。标签$0$表示未单击该标签。 该`CTRDataset`数据集还可用于加载其他数据集，如Criteo图片广告挑战赛数据集 [Dataset](https://labs.criteo.com/2014/02/kaggle-display-advertising-challenge-dataset/) 和Avazu点击率预测数据集 [Dataset](https://www.kaggle.com/c/avazu-ctr-prediction).  

## 总结
* 点击率是衡量广告系统和推荐系统有效性的重要指标。
* 点击率预测通常转化为二进制分类问题。目标是根据给定的特性预测是否会单击广告/项目。

## 练习

* 能否使用提供的`CTRDataset`加载Criteo和Avazu数据集。值得注意的是，Criteo数据集包含实值特征，因此您可能需要稍微修改代码。